In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import shapiro, kstest

pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")

Lectura de la Información:

Leer los archivos CSV (ventas.csv, productos.csv, clientes.csv).

Explorar los conjuntos de datos para comprender su estructura, columnas, tipos de datos, etc.

In [7]:
df = pd.read_csv("ventas.csv")
df.head(2)

,ID_Cliente,ID_Producto,Fecha_Venta,Cantidad,Total
0,723,A1,2023-11-22,2,17.98
1,498,C3,2023-11-21,1,5.49


In [8]:
# df2 = pd.read_csv("productos.csv",index_col=0)  #esto falla , lo vemos un poco más abajo. 

In [9]:
df3 = pd.read_csv("clientes.csv")
df3.head(2)

,id,first_name,last_name,email,gender,City,Country,Address
0,1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
1,2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction


In [10]:
import csv

input_file = "productos.csv"
output_file = "productos_corregido.csv"

with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8", newline="") as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)

    for i, row in enumerate(reader):
        if i == 0:
            writer.writerow(row)  # encabezado
            expected_columns = len(row)
        else:
            # Si hay más columnas de las esperadas, las últimas las juntamos como descripción
            if len(row) > expected_columns:
                fixed_row = row[:expected_columns - 1]
                descripcion = ",".join(row[expected_columns - 1:])
                fixed_row.append(descripcion.strip())
                writer.writerow(fixed_row)
            else:
                writer.writerow(row)

In [11]:
df4 = pd.read_csv("productos_corregido.csv",index_col=0)

In [12]:
df4.head(4)

""
ID;Nombre_Producto;Categoría;Precio;Origen;Descripción
"A1;Pizza Margherita;Platos Preparados;8.99;Italia;Clásica pizza italiana con tomate, mozzarella fresca y albahaca."
"B2;Risotto de Champiñones;Platos Preparados;6.75;Italia;Risotto cremoso con champiñones frescos, una delicia italiana para disfrutar en casa."
"C3;Tiramisú;Postres;5.49;Italia;Postre clásico italiano con capas de bizcocho de café, mascarpone y cacao en polvo."
"D4;Panettone;Repostería;10.99;Italia;Pan dulce navideño italiano con frutas confitadas y pasas, un regalo tradicional en Italia."


In [13]:
import csv

input_file = "productos.csv"
output_file = "productos_corregido2.csv"

with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8", newline="") as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)

    # Saltar líneas vacías hasta encontrar la cabecera
    for row in reader:
        if any(field.strip() for field in row):  # fila no vacía
            writer.writerow(row)  # Escribimos encabezado
            expected_columns = len(row)
            break  # salimos del loop: ya tenemos la cabecera

    # Procesar el resto del archivo
    for row in reader:
        if not any(field.strip() for field in row):  # fila vacía
            continue

        if len(row) > expected_columns:
            fixed_row = row[:expected_columns - 1]
            descripcion = ",".join(row[expected_columns - 1:])
            fixed_row.append(descripcion.strip())
            writer.writerow(fixed_row)
        else:
            writer.writerow(row)

In [14]:
df5= pd.read_csv("productos_corregido2.csv")
df5.head()

,ID;Nombre_Producto;Categoría;Precio;Origen;Descripción
0,A1;Pizza Margherita;Platos Preparados;8.99;Ita...
1,B2;Risotto de Champiñones;Platos Preparados;6....
2,C3;Tiramisú;Postres;5.49;Italia;Postre clásico...
3,D4;Panettone;Repostería;10.99;Italia;Pan dulce...
4,E5;Orecchiette;Productos Secos;4.29;Italia;Peq...


In [15]:
df2= pd.read_csv("productos.csv", sep=";")
df2.head()

,ID,Nombre_Producto,Categoría,Precio,Origen,Descripción
0,A1,Pizza Margherita,Platos Preparados,8.99,Italia,"Clásica pizza italiana con tomate, mozzarella ..."
1,B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,"Risotto cremoso con champiñones frescos, una d..."
2,C3,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...
3,D4,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...
4,E5,Orecchiette,Productos Secos,4.29,Italia,Pequeñas pastas con forma de orecchiette ideal...



Limpiar los datos: manejar valores nulos, eliminar duplicados si los hay, corregir errores tipográficos, etc.

In [16]:
df3["gender"].isna().sum()

np.int64(77)

In [17]:
df3["City"].unique()

array(['Palma De Mallorca', 'Lugo', nan, 'Leon',
       "Hospitalet De Llobregat, L'", 'Lleida', 'Santander', 'Malaga',
       'Valladolid', 'Murcia', 'Pamplona/Iruña', 'Barcelona', 'Palencia',
       'Pontevedra', 'Girona', 'Fuenlabrada', 'Jaen', 'Almeria',
       'Elx/Elche', 'Getafe', 'Sevilla', 'Coruña, A', 'Zaragoza',
       'Palmas De Gran Canaria, Las', 'Cadiz', 'Tarragona',
       'Santiago De Compostela', 'Donostia-San Sebastian', 'Cartagena',
       'Madrid', 'Ceuta', 'Vigo', 'Valencia', 'Sabadell', 'Granada',
       'Albacete', 'Teruel', 'Aviles',
       'Castellon De La Plana/Castello De La Pla', 'Vitoria-Gasteiz',
       'Zamora', 'Santa Cruz De Tenerife', 'Logroño', 'Torrevieja',
       'Leganes', 'Gijon', 'Toledo', 'Badajoz', 'Dos Hermanas', 'Ourense',
       'Ferrol', 'Salamanca', 'Sant Cugat Del Valles', 'Huesca',
       'Alicante/Alacant', 'Telde'], dtype=object)

In [18]:
nuevas_columnas = {columna: columna.lower() for columna in df3.columns}
df3.rename(columns=nuevas_columnas, inplace= True)

In [19]:
df3.head()

,id,first_name,last_name,email,gender,city,country,address
0,1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
1,2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction
2,3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing
3,4,Filmer,Eirwin,feirwin3@intel.com,NaN,Leon,Spain,5 American Ash Road
4,5,Tanhya,Lubbock,tlubbock4@huffingtonpost.com,Female,"Hospitalet De Llobregat, L'",Spain,9289 Merry Circle


In [20]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1000 non-null   int64 
 1   first_name  1000 non-null   object
 2   last_name   1000 non-null   object
 3   email       973 non-null    object
 4   gender      923 non-null    object
 5   city        876 non-null    object
 6   country     846 non-null    object
 7   address     959 non-null    object
dtypes: int64(1), object(7)
memory usage: 62.6+ KB


In [21]:
df3["city"].isna().sum()

np.int64(124)

In [22]:
df3["country"].unique()

array(['Spain', nan], dtype=object)

In [24]:
# df3["city"]["country"]

In [25]:
df2.head(2)

,ID,Nombre_Producto,Categoría,Precio,Origen,Descripción
0,A1,Pizza Margherita,Platos Preparados,8.99,Italia,"Clásica pizza italiana con tomate, mozzarella ..."
1,B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,"Risotto cremoso con champiñones frescos, una d..."


In [26]:
nuevas_columnas = {columna: columna.lower() for columna in df2.columns}
df2.rename(columns=nuevas_columnas, inplace= True)

In [27]:
nuevas_columnas = {columna: columna.lower() for columna in df.columns}
df.rename(columns=nuevas_columnas, inplace= True)

In [28]:
df.head()

,id_cliente,id_producto,fecha_venta,cantidad,total
0,723,A1,2023-11-22,2,17.98
1,498,C3,2023-11-21,1,5.49
2,121,D4,2023-11-20,3,32.97
3,885,L12,2023-11-19,1,6.49
4,347,Q17,2023-11-18,2,7.98


In [29]:
df3["id"].duplicated().sum()

np.int64(0)

In [30]:
df2["id"].duplicated().sum()

np.int64(0)

In [31]:
df3[df3.duplicated(subset='id')] #aparece vacío porque no hay duplicados
#si hubiese duplicados aparecerían en la lista. Es otra opción
#para hacer lo de la columna de arriba, pero la de arriba solo te muestra el id y sin dataframe

,id,first_name,last_name,email,gender,city,country,address


In [32]:
df["fecha_venta"].unique()

array(['2023-11-22', '2023-11-21', '2023-11-20', '2023-11-19',
       '2023-11-18', '2023-11-17', '2023-11-16', '2023-11-15',
       '2023-11-14', '2023-11-13', '2023-11-08', '2023-11-07',
       '2023-11-06', '2023-11-05', '2023-11-04', '2023-11-03',
       '2023-11-02', '2023-11-01', '2023-10-31', '2023-10-30',
       '2023-10-29', '2023-10-28', '2023-10-27', '2023-10-26',
       '2023-10-25', '2023-10-24', '2023-10-23', '2023-10-22',
       '2023-10-21', '2023-10-20', '2023-10-19', '2023-10-18',
       '2023-10-17', '2023-10-16', '2023-10-15', '2023-10-14',
       '2023-10-13', '2023-10-12', '2023-10-11', '2023-10-10',
       '2023-10-09', '2023-10-08', '2023-10-07', '2023-10-06',
       '2023-10-05', '2023-10-04', '2023-10-03', '2023-10-02',
       '2023-10-01', '2023-09-30', '2023-09-29', '2023-09-28',
       '2023-09-27', '2023-09-26', '2023-09-25', '2023-09-24',
       '2023-09-23', '2023-09-22', '2023-09-21', '2023-09-20',
       '2023-09-19', '2023-09-18', '2023-09-17', '2023-


Realizar la integración de datos: unir los conjuntos de datos apropiados para obtener una tabla única que contenga información de ventas junto con detalles de productos y clientes.

In [34]:
#df es ventas
#añadir detalles de descripción?
df

# Realizar la integración de datos: unir los conjuntos de datos apropiados para obtener una tabla única que contenga información de ventas junto con detalles de productos y clientes.

,id_cliente,id_producto,fecha_venta,cantidad,total
0,723,A1,2023-11-22,2,17.98
1,498,C3,2023-11-21,1,5.49
2,121,D4,2023-11-20,3,32.97
3,885,L12,2023-11-19,1,6.49
4,347,Q17,2023-11-18,2,7.98
...,...,...,...,...,...
95,234,AR136,2023-09-04,1,8.45
96,987,AS137,2023-09-03,2,12.50
97,543,AT138,2023-09-02,1,4.75
98,234,AU139,2023-09-01,2,9.98


In [37]:
df.head(1)

,id_cliente,id_producto,fecha_venta,cantidad,total
0,723,A1,2023-11-22,2,17.98


In [41]:
# Paso 1: Unir ventas con clientes
ventas_clientes = pd.merge(
   df,
    df3,
    how='left',
    left_on='id_cliente',
    right_on='id'
)

In [43]:

# Paso 2: Unir el resultado con productos
df_final = pd.merge(
    ventas_clientes,
    df2,
    how='left',
    left_on='id_producto',
    right_on='id'
)

In [40]:
df2.head(1)

,id,nombre_producto,categoría,precio,origen,descripción
0,A1,Pizza Margherita,Platos Preparados,8.99,Italia,"Clásica pizza italiana con tomate, mozzarella ..."


In [39]:
df3.head(1)

,id,first_name,last_name,email,gender,city,country,address
0,1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing


In [46]:
df_final.columns

Index(['id_cliente', 'id_producto', 'fecha_venta', 'cantidad', 'total', 'id_x',
       'first_name', 'last_name', 'email', 'gender', 'city', 'country',
       'address', 'id_y', 'nombre_producto', 'categoría', 'precio', 'origen',
       'descripción'],
      dtype='object')

In [47]:


# df ventas  todo 
# df2 productos nombre_producto categoria precio origen descripcion (no incluir id)
# df3 cliente todo

# ventas junto con detalles de productos y clientes

""" Si después del merge tienes columnas repetidas como cliente_id y id_cliente, puedes limpiarlas:
python
CopiarEditar """
# Eliminar columnas duplicadas o renombrar
df_final.drop(['id_y', 'id_x'], axis=1, inplace=True)



In [48]:
df_final.head(1)

,id_cliente,id_producto,fecha_venta,cantidad,total,first_name,last_name,email,gender,city,country,address,nombre_producto,categoría,precio,origen,descripción
0,723,A1,2023-11-22,2,17.98,Sephira,Bleackley,sbleackleyk2@liveinternet.ru,NaN,Girona,NaN,4 Birchwood Road,Pizza Margherita,Platos Preparados,8.99,Italia,"Clásica pizza italiana con tomate, mozzarella ..."


In [50]:
df_final.columns

Index(['id_cliente', 'id_producto', 'fecha_venta', 'cantidad', 'total',
       'first_name', 'last_name', 'email', 'gender', 'city', 'country',
       'address', 'nombre_producto', 'categoría', 'precio', 'origen',
       'descripción'],
      dtype='object')

In [49]:
df_final.to_csv('df_pair.csv', index=False)

In [ ]:
import pandas as pd
# Lê apenas as colunas desejadas de cada arquivo
df1 = pd.read_csv('arquivo1.csv', usecols=['id', 'email'])
df2 = pd.read_csv('arquivo2.csv', usecols=['id', 'city'])
df3 = pd.read_csv('arquivo3.csv', usecols=['id', 'country'])
# Junta todos com base no id (merge horizontal, não vertical)
df_total = df1.merge(df2, on='id', how='outer') \
              .merge(df3, on='id', how='outer')
# Visualiza o resultado
print(df_total.head())

In [ ]:
import pandas as pd
# Lê os 3 arquivos
df1 = pd.read_csv('arquivo1.csv')
df2 = pd.read_csv('arquivo2.csv')
df3 = pd.read_csv('arquivo3.csv')
# Junta todos em um único DataFrame
df_total = pd.concat([df1, df2, df3], ignore_index=True)
# Visualiza o resultado
print(df_total.head())